# Import

In [ ]:
import pandas as pd


item_categories = pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')
items = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')
sales_train = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')
shops = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')
test = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')

# Feature engineering

date column - change string into timestamp

In [ ]:
type(sales_train['date'][0])

In [ ]:
from datetime import datetime
  
def timestamp(x):
    return datetime.strptime(x, "%d.%m.%Y")

In [ ]:
sales_train['date'] = sales_train['date'].apply(timestamp)

In [ ]:
type(sales_train['date'][0])

In [ ]:
sales_train

add item_category column

In [ ]:
items.drop('item_name',1,inplace = True)
items

sort by date

In [ ]:
sales_train.sort_values(by = 'date', inplace = True)
sales_train.reset_index(drop = True, inplace = True)
sales_train

reorganize columns

In [ ]:
sales_train = sales_train[['shop_id','item_id','date','date_block_num','item_price','item_cnt_day']]
sales_train.reset_index(drop = True, inplace = True)
sales_train

pivot table

In [ ]:
dataset = sales_train.pivot_table(index = ['shop_id','item_id'],values = ['item_cnt_day'],columns = ['date_block_num'], fill_value= 0)
dataset.reset_index(inplace = True)
dataset

merge with test data

In [ ]:
test

In [ ]:
dataset = pd.merge(test, dataset, on = ['shop_id','item_id'], how = 'left')
dataset

fill nan with 0

In [ ]:
dataset.fillna(0, inplace = True)
dataset

drop shop_id and item_id. Since ID corresponds to index we will drop it as well.

In [ ]:
dataset.drop(labels = ['shop_id','ID','item_id'],inplace = True,axis = 1)
dataset

# Training the model

train test split

Time series needs same amount of independent varaibles to predict the model as the amount that model was trained on, that's why X_train columns are [0,32] and X_test are [1,33]

In [ ]:
X_train = dataset.iloc[:,:-1]
y_train = dataset.iloc[:,-1:]
X_test = dataset.iloc[:,1:]
print(X_train.shape, y_train.shape, X_test.shape)

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
X_test

# Model

In [ ]:
from sklearn.linear_model import LinearRegression

regression = LinearRegression()

regression.fit(X_train, y_train)

In [ ]:
prediction = regression.predict(X_test)
prediction

In [ ]:
y_train.shape

In [ ]:
from sklearn import linear_model

clf = linear_model.Lasso(alpha=0.00001)

clf.fit(X_train,y_train)

predictions_train = clf.predict(X_train)
predictions_train

# Score

In [ ]:
clf.score(X_train, y_train)

# Final prediction

In [ ]:
predictions = clf.predict(X_test)
predictions

# Submission

In [ ]:
sample_submission = pd.read_csv('../input/competitive-data-science-predict-future-sales/sample_submission.csv')

sample_submission['item_cnt_month'] = predictions
sample_submission

In [ ]:
sample_submission.to_csv('ss.csv',index = False)

In [ ]:
pd.read_csv('ss.csv')